In [1]:
import random
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score,StratifiedKFold,learning_curve,train_test_split
from PIL import ImageFile                            
ImageFile.LOAD_TRUNCATED_IMAGES = True                 
import os

In [2]:
import os
import sys
import numpy as np
import cv2
import csv
import pandas as pd
from tqdm import tqdm
from pandas import Series, DataFrame
from keras.preprocessing import image
from pandas import Series, DataFrame

Using TensorFlow backend.


In [3]:
def path_to_tensor(path_name):
    # loads RGB image as PIL.Image.Image type
    img_path = path_name;
    img = image.load_img(img_path, target_size=(224, 224))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    aa = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(aa, axis=0)


def read_path(file_path):
    list_of_tensors = [path_to_tensor(i) for i in tqdm(file_path)]
    return np.vstack(list_of_tensors)

In [4]:
dic = {
    "CALsuburb":9,
"PARoffice":7,
"bedroom":12,
"coast":10,
"forest":4,
"highway":14,
"industrial":2,
"insidecity":3,
"kitchen":0,
"livingroom":5,
"mountain":8,
"opencountry":6,
"store":11,
"street":1,
"tallbuilding":13,
}

# training set

In [5]:
train_path = os.getcwd() + '\\Q2\\train'
train_info = []
fold_path = os.listdir(train_path)
for fold in fold_path:
    file_name = os.listdir(os.path.join(train_path,fold))
    for f in file_name:
        train_info.append([os.path.join(train_path,fold,f),fold,dic[fold]])
df_train_info = pd.DataFrame(train_info,columns = ['file_path','class','label'])


In [6]:
from tensorflow.keras.applications.resnet50 import preprocess_input as preprocess_input_resnet50
from keras.utils import np_utils

train_images = read_path(df_train_info['file_path'].tolist())
train_labels = df_train_info['label'].tolist()

x_train,x_valid,y_train,y_valid = train_test_split(train_images,train_labels,test_size = 0.2,random_state = 42)

train_images = x_train.astype('float32')
train_images = preprocess_input_resnet50(train_images)
train_labels = np_utils.to_categorical(y_train, 15)    

valid_images = x_valid.astype('float32')
valid_images = preprocess_input_resnet50(valid_images)
valid_labels = np_utils.to_categorical(y_valid, 15)    

print("train image shape: ")
print(train_images.shape)
print("valid image shape: ")
print(valid_images.shape)


100%|████████████████████████████████████████████████████████████████████████████| 2985/2985 [00:02<00:00, 1069.26it/s]


train image shape: 
(2388, 224, 224, 3)
valid image shape: 
(597, 224, 224, 3)


# Create Model

In [9]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Conv2D,Flatten,Dense,Activation,GlobalAveragePooling2D
from tensorflow.keras import regularizers

resmodel = ResNet50(weights='imagenet',include_top = False,input_shape = (224,224,3))

#for layer in resmodel.layers:
#    layer.trainable = True
    
y = resmodel.output
y = GlobalAveragePooling2D()(y)
y = Dense(1024,kernel_regularizer = regularizers.l2(0.001),activation = 'relu')(y)
y = Dense(15,activation='softmax')(y)
model = Model(inputs=resmodel.input,outputs=y)
model.summary()

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
_______________________________________________________________________________________

# Training

In [10]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import Adam,Adamax
epochs = 1000
sgd = SGD(lr = 0.01,decay = 1e-6,momentum=0.9,nesterov=True)
model.compile(loss='categorical_crossentropy' , optimizer = sgd, metrics=['accuracy'])
checkpointer = ModelCheckpoint(filepath = 'test.hdf5',verbose = 1,save_best_only = True)

model.fit(train_images,train_labels,validation_data = (valid_images,valid_labels),epochs = epochs,
         batch_size = 64,callbacks = [checkpointer],verbose = 1 )


Epoch 1/1000
38/38 [==============================] - ETA: 0s - loss: 2.0833 - accuracy: 0.7730
Epoch 00001: val_loss improved from inf to 2.83651, saving model to test.hdf5
38/38 [==============================] - 26s 687ms/step - loss: 2.0833 - accuracy: 0.7730 - val_loss: 2.8365 - val_accuracy: 0.6951
Epoch 2/1000
38/38 [==============================] - ETA: 0s - loss: 1.4825 - accuracy: 0.9539
Epoch 00002: val_loss did not improve from 2.83651
38/38 [==============================] - 25s 650ms/step - loss: 1.4825 - accuracy: 0.9539 - val_loss: 2.9589 - val_accuracy: 0.7353
Epoch 3/1000
38/38 [==============================] - ETA: 0s - loss: 1.3648 - accuracy: 0.9891
Epoch 00003: val_loss improved from 2.83651 to 1.88498, saving model to test.hdf5
38/38 [==============================] - 25s 667ms/step - loss: 1.3648 - accuracy: 0.9891 - val_loss: 1.8850 - val_accuracy: 0.8509
Epoch 4/1000
38/38 [==============================] - ETA: 0s - loss: 1.3169 - accuracy: 0.9966
Epoch 000

38/38 [==============================] - ETA: 0s - loss: 0.9074 - accuracy: 1.0000
Epoch 00028: val_loss improved from 1.14054 to 1.12621, saving model to test.hdf5
38/38 [==============================] - 26s 673ms/step - loss: 0.9074 - accuracy: 1.0000 - val_loss: 1.1262 - val_accuracy: 0.9414
Epoch 29/1000
38/38 [==============================] - ETA: 0s - loss: 0.8935 - accuracy: 1.0000
Epoch 00029: val_loss improved from 1.12621 to 1.09678, saving model to test.hdf5
38/38 [==============================] - 26s 672ms/step - loss: 0.8935 - accuracy: 1.0000 - val_loss: 1.0968 - val_accuracy: 0.9414
Epoch 30/1000
38/38 [==============================] - ETA: 0s - loss: 0.8808 - accuracy: 0.9996
Epoch 00030: val_loss did not improve from 1.09678
38/38 [==============================] - 25s 652ms/step - loss: 0.8808 - accuracy: 0.9996 - val_loss: 1.1141 - val_accuracy: 0.9430
Epoch 31/1000
38/38 [==============================] - ETA: 0s - loss: 0.8705 - accuracy: 0.9987
Epoch 00031: va

Epoch 55/1000
38/38 [==============================] - ETA: 0s - loss: 0.6022 - accuracy: 1.0000
Epoch 00055: val_loss improved from 0.79893 to 0.78997, saving model to test.hdf5
38/38 [==============================] - 26s 676ms/step - loss: 0.6022 - accuracy: 1.0000 - val_loss: 0.7900 - val_accuracy: 0.9514
Epoch 56/1000
38/38 [==============================] - ETA: 0s - loss: 0.5931 - accuracy: 1.0000
Epoch 00056: val_loss improved from 0.78997 to 0.78456, saving model to test.hdf5
38/38 [==============================] - 26s 672ms/step - loss: 0.5931 - accuracy: 1.0000 - val_loss: 0.7846 - val_accuracy: 0.9464
Epoch 57/1000
38/38 [==============================] - ETA: 0s - loss: 0.5842 - accuracy: 1.0000
Epoch 00057: val_loss improved from 0.78456 to 0.77901, saving model to test.hdf5
38/38 [==============================] - 26s 676ms/step - loss: 0.5842 - accuracy: 1.0000 - val_loss: 0.7790 - val_accuracy: 0.9447
Epoch 58/1000
38/38 [==============================] - ETA: 0s - lo

Epoch 82/1000
38/38 [==============================] - ETA: 0s - loss: 0.4002 - accuracy: 1.0000
Epoch 00082: val_loss improved from 0.61336 to 0.60495, saving model to test.hdf5
38/38 [==============================] - 26s 676ms/step - loss: 0.4002 - accuracy: 1.0000 - val_loss: 0.6049 - val_accuracy: 0.9414
Epoch 83/1000
38/38 [==============================] - ETA: 0s - loss: 0.3944 - accuracy: 1.0000
Epoch 00083: val_loss improved from 0.60495 to 0.59918, saving model to test.hdf5
38/38 [==============================] - 26s 673ms/step - loss: 0.3944 - accuracy: 1.0000 - val_loss: 0.5992 - val_accuracy: 0.9414
Epoch 84/1000
38/38 [==============================] - ETA: 0s - loss: 0.3886 - accuracy: 1.0000
Epoch 00084: val_loss improved from 0.59918 to 0.58904, saving model to test.hdf5
38/38 [==============================] - 25s 667ms/step - loss: 0.3886 - accuracy: 1.0000 - val_loss: 0.5890 - val_accuracy: 0.9430
Epoch 85/1000
38/38 [==============================] - ETA: 0s - lo

Epoch 109/1000
38/38 [==============================] - ETA: 0s - loss: 0.2663 - accuracy: 1.0000
Epoch 00109: val_loss improved from 0.47796 to 0.47219, saving model to test.hdf5
38/38 [==============================] - 25s 660ms/step - loss: 0.2663 - accuracy: 1.0000 - val_loss: 0.4722 - val_accuracy: 0.9397
Epoch 110/1000
38/38 [==============================] - ETA: 0s - loss: 0.2623 - accuracy: 1.0000
Epoch 00110: val_loss improved from 0.47219 to 0.46663, saving model to test.hdf5
38/38 [==============================] - 25s 663ms/step - loss: 0.2623 - accuracy: 1.0000 - val_loss: 0.4666 - val_accuracy: 0.9447
Epoch 111/1000
38/38 [==============================] - ETA: 0s - loss: 0.2584 - accuracy: 1.0000
Epoch 00111: val_loss improved from 0.46663 to 0.46162, saving model to test.hdf5
38/38 [==============================] - 25s 667ms/step - loss: 0.2584 - accuracy: 1.0000 - val_loss: 0.4616 - val_accuracy: 0.9464
Epoch 112/1000
38/38 [==============================] - ETA: 0s 

Epoch 136/1000
38/38 [==============================] - ETA: 0s - loss: 0.1775 - accuracy: 1.0000
Epoch 00136: val_loss did not improve from 0.37632
38/38 [==============================] - 25s 648ms/step - loss: 0.1775 - accuracy: 1.0000 - val_loss: 0.3864 - val_accuracy: 0.9397
Epoch 137/1000
38/38 [==============================] - ETA: 0s - loss: 0.1748 - accuracy: 1.0000
Epoch 00137: val_loss did not improve from 0.37632
38/38 [==============================] - 25s 650ms/step - loss: 0.1748 - accuracy: 1.0000 - val_loss: 0.3781 - val_accuracy: 0.9414
Epoch 138/1000
38/38 [==============================] - ETA: 0s - loss: 0.1723 - accuracy: 1.0000
Epoch 00138: val_loss improved from 0.37632 to 0.37008, saving model to test.hdf5
38/38 [==============================] - 25s 668ms/step - loss: 0.1723 - accuracy: 1.0000 - val_loss: 0.3701 - val_accuracy: 0.9414
Epoch 139/1000
38/38 [==============================] - ETA: 0s - loss: 0.1697 - accuracy: 1.0000
Epoch 00139: val_loss improv

Epoch 164/1000
38/38 [==============================] - ETA: 0s - loss: 0.1171 - accuracy: 1.0000
Epoch 00164: val_loss improved from 0.33243 to 0.33034, saving model to test.hdf5
38/38 [==============================] - 25s 666ms/step - loss: 0.1171 - accuracy: 1.0000 - val_loss: 0.3303 - val_accuracy: 0.9447
Epoch 165/1000
38/38 [==============================] - ETA: 0s - loss: 0.1154 - accuracy: 1.0000
Epoch 00165: val_loss improved from 0.33034 to 0.32620, saving model to test.hdf5
38/38 [==============================] - 25s 669ms/step - loss: 0.1154 - accuracy: 1.0000 - val_loss: 0.3262 - val_accuracy: 0.9447
Epoch 166/1000
38/38 [==============================] - ETA: 0s - loss: 0.1136 - accuracy: 1.0000
Epoch 00166: val_loss did not improve from 0.32620
38/38 [==============================] - 25s 649ms/step - loss: 0.1136 - accuracy: 1.0000 - val_loss: 0.3283 - val_accuracy: 0.9430
Epoch 167/1000
38/38 [==============================] - ETA: 0s - loss: 0.1119 - accuracy: 1.00

KeyboardInterrupt: 

# Predict

In [11]:
model.load_weights('test.hdf5')

In [16]:
test_info = []
test_path = os.getcwd() + '\\Q2\\test'
file_name = os.listdir(test_path)
test_info = [os.path.join(test_path,f) for f in file_name]


In [17]:
test_images = read_path(test_info)
test_images = test_images.astype('float32')
test_images = preprocess_input_resnet50(test_images)

print("test image shape: ")
print(test_images.shape)


100%|█████████████████████████████████████████████████████████████████████████████| 1500/1500 [00:08<00:00, 184.57it/s]


test image shape: 
(1500, 224, 224, 3)


In [21]:
pred_y = model.predict(test_images)
pred_y = [np.argmax(p) for p in pred_y]

In [23]:
df_temp = pd.read_csv('img-submission.csv')

In [27]:
submit = pd.DataFrame({"id":df_temp['id'],"class":pred_y})

In [29]:
submit.to_csv('q2_submit.csv',index = False)